In [1]:
import os
import dotenv

# Load base path
dotenv.load_dotenv(dotenv.find_dotenv())
path = os.getenv("IMAGES_PATH")
test_path = os.getenv("TEST_IMAGES_PATH")

# Get all images in the path
def getImages(base_path):
    imgList = []

    # Iterate through all files in the path
    for filename in os.listdir(base_path):

        # Check if the file is an image
        if filename.endswith(".png") or filename.endswith(".jpg"):
            print(base_path)
            # Add it to the list if so
            imgList.append(os.path.join(base_path, filename))
        
    # return the list of filepaths
    return imgList

In [2]:
import cv2
import os

# Function to load image and preprocess it
def cutImage(image_path):
    videos = []  # List to hold the cut images

    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Use adaptive thresholding to identify the regions of the image that contain content
    adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    
    # Find contours of the regions
    contours, _ = cv2.findContours(adaptive_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter and sort the contours
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 5000]
    filtered_contours.sort(key=lambda x: cv2.boundingRect(x)[1])

    # Loop through each contour
    # Cut the images along the y-axis and add them to the list
    for cnt in filtered_contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if  w > h: 
            # Cut the image along the y-axis while keeping the full width of the image
            full_video_item = img[y:y+h, 0:img.shape[1]]
            videos.append(full_video_item)

    # Return the list of cut images
    return videos

In [3]:
import pytesseract
# https://github.com/tesseract-ocr/tesseract is required!!!

# Call the function to cut the image into separate video items
cut_images = cutImage(test_path)

# Save the cut images for review
saved_paths = []
for index, image in enumerate(cut_images):
    print(text = pytesseract.image_to_string(image, lang='eng'))
    save_path = os.path.join(path, f'cut/video_item_{index}.png')
    cv2.imwrite(save_path, image)
    saved_paths.append(save_path)

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.